## Import Libraries and Dataframes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU 
LeakyReLU = LeakyReLU(negative_slope=0.1)


In [4]:
# path variable 
path = r'C:\Users\jmiel\JENNY_STUFF\MY_DOCS\CareerFoundry\ClimateWins'

# Upload weather file
climate_cleaned = pd.read_csv(os.path.join(path, 'Data Sets', 'climate_cleaned.csv'))
climate_cleaned

,Unnamed: 0,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


In [6]:
climate_cleaned = climate_cleaned.drop('Unnamed: 0', axis =1)

In [8]:
pleasant_weather = pd.read_csv(os.path.join(path, 'Data Sets', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
pleasant_weather

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,20221028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,20221029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,20221030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
pleasant_weather.drop(columns = 'DATE', inplace = True)

In [14]:
pleasant_weather.shape

(22950, 15)

In [24]:
# Turn X and answers from a df to arrays

X = np.array(climate_cleaned)
y = np.array(pleasant_weather)

In [26]:
X = X.reshape(-1,15,9)

In [28]:
# Verify shape

X.shape

(22950, 15, 9)

In [30]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [32]:
y.shape

(22950,)

In [36]:
# Check y layout
type_of_target(y)

'multiclass'

In [38]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [40]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


## Bayesian Search Hyperparameter Optimization for CNN

In [44]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [46]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [48]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\jmiel\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
C:\Users\jmiel\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\jmiel\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
15/15 - 3s - 215ms/step - accuracy: 0.6001 - loss: 2.7111
Epoch 2/25
15/15 - 0s - 21ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 0s - 21ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6853
Epoch 9/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 0s - 21ms/step - accuracy: 0.6440 - loss: 2.6783
Epoch 13/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6767
Epoch 14/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6752
Epoch 15/25
15/15 - 0s - 20ms/step - accur

C:\Users\jmiel\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\jmiel\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 3s - 203ms/step - accuracy: 0.6022 - loss: 2.6997
Epoch 2/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6854
Epoch 9/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6784
Epoch 13/25
15/15 - 0s - 21ms/step - accuracy: 0.6440 - loss: 2.6768
Epoch 14/25
15/15 - 0s - 20ms/step - accuracy: 0.6440 - loss: 2.6753
Epoch 15/25
15/15 - 0s - 20ms/step - accuracy: 0.6440

...

C:\Users\jmiel\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\jmiel\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 3s - 133ms/step - accuracy: 0.5330 - loss: 1.5195
Epoch 2/35
21/21 - 0s - 9ms/step - accuracy: 0.6784 - loss: 0.9797
Epoch 3/35
21/21 - 0s - 9ms/step - accuracy: 0.6950 - loss: 0.8981
Epoch 4/35
21/21 - 0s - 9ms/step - accuracy: 0.7097 - loss: 0.8431
Epoch 5/35
21/21 - 0s - 9ms/step - accuracy: 0.7278 - loss: 0.7957
Epoch 6/35
21/21 - 0s - 9ms/step - accuracy: 0.7418 - loss: 0.7504
Epoch 7/35
21/21 - 0s - 9ms/step - accuracy: 0.7544 - loss: 0.7107
Epoch 8/35
21/21 - 0s - 9ms/step - accuracy: 0.7630 - loss: 0.6760
Epoch 9/35
21/21 - 0s - 9ms/step - accuracy: 0.7784 - loss: 0.6394
Epoch 10/35
21/21 - 0s - 9ms/step - accuracy: 0.7895 - loss: 0.6033
Epoch 11/35
21/21 - 0s - 9ms/step - accuracy: 0.8039 - loss: 0.5661
Epoch 12/35
21/21 - 0s - 9ms/step - accuracy: 0.8122 - loss: 0.5361
Epoch 13/35
21/21 - 0s - 9ms/step - accuracy: 0.8159 - loss: 0.5092
Epoch 14/35
21/21 - 0s - 9ms/step - accuracy: 0.8340 - loss: 0.4741
Epoch 15/35
21/21 - 0s - 9ms/step - accuracy: 0.8391 - loss: 0.452

C:\Users\jmiel\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\jmiel\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 3s - 129ms/step - accuracy: 0.4998 - loss: 1.6847
Epoch 2/35
21/21 - 0s - 8ms/step - accuracy: 0.6519 - loss: 1.0206
Epoch 3/35
21/21 - 0s - 9ms/step - accuracy: 0.6757 - loss: 0.9273
Epoch 4/35
21/21 - 0s - 9ms/step - accuracy: 0.6965 - loss: 0.8651
Epoch 5/35
21/21 - 0s - 9ms/step - accuracy: 0.7176 - loss: 0.8094
Epoch 6/35
21/21 - 0s - 9ms/step - accuracy: 0.7347 - loss: 0.7578
Epoch 7/35
21/21 - 0s - 9ms/step - accuracy: 0.7471 - loss: 0.7157
Epoch 8/35
21/21 - 0s - 9ms/step - accuracy: 0.7609 - loss: 0.6761
Epoch 9/35
21/21 - 0s - 9ms/step - accuracy: 0.7755 - loss: 0.6348
Epoch 10/35
21/21 - 0s - 9ms/step - accuracy: 0.7935 - loss: 0.5849
Epoch 11/35
21/21 - 0s - 9ms/step - accuracy: 0.8110 - loss: 0.5297
Epoch 12/35
21/21 - 0s - 9ms/step - accuracy: 0.8289 - loss: 0.4861
Epoch 13/35
21/21 - 0s - 9ms/step - accuracy: 0.8465 - loss: 0.4415
Epoch 14/35
21/21 - 0s - 9ms/step - accuracy: 0.8572 - loss: 0.4120
Epoch 15/35
21/21 - 0s - 9ms/step - accuracy: 0.8748 - loss: 0.369

ValueError: Input y contains NaN.

In [50]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x19a1a3c9ad0>}

In [52]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])  
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  
neurons = 61
normalization = 0.770967179954561  
dropout = 0.7296061783380641  
dropout_rate = 0.19126724140656393  
optimizer = Adadelta(learning_rate=0.7631771981307285)  

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

# Add second set of dense layers
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())  # Max-pooling layer to downsample the data
model.add(Flatten())  # Flatten for the dense layers
model.add(Dense(n_classes, activation='softmax'))  # Output layer for classification

# Compile the model with specified loss and optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\jmiel\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [56]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [58]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [60]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 3s - 81ms/step - accuracy: 0.6120 - loss: 1.3987
Epoch 2/47
38/38 - 0s - 9ms/step - accuracy: 0.6902 - loss: 0.9103
Epoch 3/47
38/38 - 0s - 9ms/step - accuracy: 0.7079 - loss: 0.8390
Epoch 4/47
38/38 - 0s - 9ms/step - accuracy: 0.7253 - loss: 0.7875
Epoch 5/47
38/38 - 0s - 9ms/step - accuracy: 0.7429 - loss: 0.7465
Epoch 6/47
38/38 - 0s - 9ms/step - accuracy: 0.7572 - loss: 0.7088
Epoch 7/47
38/38 - 0s - 9ms/step - accuracy: 0.7720 - loss: 0.6732
Epoch 8/47
38/38 - 0s - 9ms/step - accuracy: 0.7807 - loss: 0.6447
Epoch 9/47
38/38 - 0s - 9ms/step - accuracy: 0.7887 - loss: 0.6195
Epoch 10/47
38/38 - 0s - 9ms/step - accuracy: 0.7991 - loss: 0.5837
Epoch 11/47
38/38 - 0s - 9ms/step - accuracy: 0.8050 - loss: 0.5657
Epoch 12/47
38/38 - 0s - 9ms/step - accuracy: 0.8147 - loss: 0.5385
Epoch 13/47
38/38 - 0s - 9ms/step - accuracy: 0.8204 - loss: 0.5259
Epoch 14/47
38/38 - 0s - 9ms/step - accuracy: 0.8295 - loss: 0.4937
Epoch 15/47
38/38 - 0s - 9ms/step - accuracy: 0.8313 - l

## Creating a Confusion Matrix

In [64]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [80]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [82]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test),stations))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3475       150        15       7           5         5       1   
BELGRADE       77       964        22       2           1         5       0   
BUDAPEST       19         5       163       8           2         7       0   
DEBILT          8         4         3      57           0         4       0   
DUSSELDORF      2         0         0       2           6        10       0   
HEATHROW       10         0         0       1           1        48       0   
KASSEL          1         1         0       0           0         0       5   
LJUBLJANA       7         2         0       0           0         0       0   
MAASTRICHT      6         0         0       0           0         0       0   
MADRID         34         3         3       0           0         2       0   
MUNCHENB   